# CSS

In [1]:
from IPython.display import HTML
style = """
<style>
.expo {
  line-height: 150%;
}

.visual {
  width: 400px;
}

</style>
"""
HTML(style)

# Putting this all together

Now let's write some functions that train the neural net. The following just wraps around what we've already done, running one batch through the neural net:

In [24]:
def learn(V, W, x_batch, y_batch):
    # forward pass
    A = np.dot(x_batch,V)
    B = sigmoid(A)
    C = np.dot(B,W)
    P = sigmoid(C)
    
    # loss
    L = 0.5 * (y_batch - P) ** 2
    
    # backpropogation
    dLdP = -1.0 * (y_batch - P)
    dPdC = sigmoid(C) * (1-sigmoid(C))
    dLdC = dLdP * dPdC
    dCdW = B.T
    dLdW = np.dot(dCdW, dLdC)
    dCdB = W.T
    dBdA = sigmoid(A) * (1-sigmoid(A))
    dAdV = x.T
    dLdV = np.dot(dAdV, np.dot(dLdP * dPdC, dCdB) * dBdA)
    
    # update the weights
    W -= dLdW
    V -= dLdV
    
    return V, W

### Putting this all together, Pt. 1

Now a function to initialize the weights:

In [36]:
def initialize_weights(num_hidden=4):
    '''
    Randomly initializes weights
    '''
    np.random.seed(1104)
    V = np.random.randn(3, num_hidden)
    W = np.random.randn(num_hidden, 1)
    return V, W

### Putting this all together, Pt. 2

And one to shuffle the data:

In [26]:
def shuffle_x_y(X, Y):
    '''
    Each array must be two dimensional
    '''
    np.random.seed(1104)
    train_size = X.shape[0]
    indices = list(range(train_size))
    np.random.shuffle(indices)
    return X[indices], Y[indices]

### Putting this all together, Pt. 3

This runs the data through the net, one observation at a time:

In [27]:
def one_epoch(X, Y, V, W):
    '''
    Run one epoch an element at a time through the net.
    '''
    for index in range(X.shape[0]):
        x_batch = np.array(X[index], ndmin=2)
        y_batch = np.array(Y[index], ndmin=2)
        learn(V, W, x_batch, y_batch)
    return V, W

### Putting this all together, Pt. 4

This takes in a batch of observations and makes predictions using the latest learned weights:

In [28]:
def predict(x_batch, V, W):
    '''
    Make a prediction given a batch of observations and the weights.
    '''
    A = np.dot(x_batch,V)
    B = sigmoid(A)
    C = np.dot(B,W)
    P = sigmoid(C)
    return P

### Putting this all together, Pt. 5

This calculates the loss:

In [29]:
def loss(prediction, actual, print_loss=False):
    '''
    Calculate the loss.
    '''
    df = pd.DataFrame(data=np.concatenate([prediction, actual], axis=1),
                      columns=['predictions', 'actuals'])
    df['loss'] = 0.5 * (df['predictions'] - df['actuals']) ** 2
    return df['loss'].mean()

### Putting this all together, Pt. 6

Finally, this function trains the net for a number of epochs and outputs a data frame illustrating the training progress:

In [30]:
def train(X, Y, V, W, epochs=100):
    '''
    Train the net for a number of epochs.
    '''
    losses = []
    epochs_list = []
    for i in range(epochs+1):
        V, W = one_epoch(X, Y, V, W)
        if i % (epochs / 10) == 0:
            preds = predict(X, V, W)
            loss_epoch = loss(preds, Y)
            epochs_list.append(i)
            losses.append(loss_epoch)
    return pd.DataFrame({'epoch' : epochs_list, 
                         'loss' : losses})

### Putting this all together, Pt. 7

This function wraps around much of what we've done so far, training the net and displaying the result of the training:

In [31]:
def train_and_display(X, Y, num_epochs=1000, num_hidden=8):
    X, Y = shuffle_x_y(X, Y)
    V, W = initialize_weights(num_hidden=num_hidden)
    df = train(X, Y, V, W, num_epochs)
    df_print(df)
    return V, W

### Putting this all together, Pt. 8

And this monster of a function computes the accuracy of the net:

In [39]:
def accuracy(X, Y, V, W):
    
    def _df_actual_predicted(X, Y, V, W):
        return pd.DataFrame(np.round(np.concatenate([Y, predict(X, V, W)], axis=1), 2),
                            columns=["Actual", "Predicted"])

    df = _df_actual_predicted(X, Y, V, W)
    print("The data frame of the predictions this neural net produces is:\n",
          df)
    df['Prediction'] = df['Predicted'] > 0.5

    def _correct_prediction(row):
        return bool(row['Actual']) == row['Prediction']
    
    df['Correct'] = df.apply(lambda x: _correct_prediction(x), axis=1)
        
    print("The accuracy of this trained neural net is", 
          df['Correct'].sum() / len(df['Correct']))
    
    return df['Correct'].sum() / len(df['Correct'])

### Putting this all together, Pt. 9

Now, let's play around with some results:

In [40]:
V, W = train_and_display(X, Y, 1000, 8)
accuracy(X, Y, V, W)

    epoch  loss
0       0  0.22
1     100  0.05
2     200  0.04
3     300  0.04
4     400  0.03
5     500  0.03
6     600  0.02
7     700  0.02
8     800  0.02
9     900  0.02
10   1000  0.01
The data frame of the predictions this neural net produces is:
    Actual  Predicted
0     1.0       0.67
1     0.0       0.20
2     1.0       1.00
3     0.0       0.00
4     0.0       0.13
5     0.0       0.00
6     1.0       0.99
7     1.0       0.76
The accuracy of this trained neural net is 1.0


1.0